In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout

In [3]:
# Load the uploaded dataset
file_path = '/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan_real.csv'
dataset = pd.read_csv(file_path)
dataset = dataset.iloc[:, 0].str.split(';', expand=True)
dataset.columns = [
    'No_Induk', 'Name', 'DOB', 'Gender', 'Age',
    'Weight', 'Height', 'Check_up_month', 'Check_up_year'
]

dataset

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Check_up_month,Check_up_year
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81,Februari,2022
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10,75,Februari,2022
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9,76,Februari,2022
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96,Februari,2022
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88,Februari,2022
...,...,...,...,...,...,...,...,...,...
826,46,Sena Humaira Aviaji,04/11/2024,Perempuan,17,8.2,75,Mei,2025
827,47,Nabila Putri Calista,29/05/2024,Perempuan,17,9.1,74.2,Mei,2025
828,48,Kanaya Zunaira Mafaza,18/06/2024,Perempuan,19,8.9,78.1,Mei,2025
829,49,Muh. Davindra Rizky Pratama,29/07/2024,Laki-laki,19,9,73.7,Mei,2025


In [4]:
bulan_map = {
    "Januari": "01", "Februari": "02", "Maret": "03", "April": "04", "Mei": "05",
    "Juni": "06", "Juli": "07", "Agustus": "08", "September": "09",
    "Oktober": "10", "November": "11", "Desember": "12"
}

# Ubah nama bulan ke angka
dataset["Month_num"] = dataset["Check_up_month"].map(bulan_map)

# Buat kolom datetime (dengan tanggal default 1)
dataset["Checkup_date"] = pd.to_datetime(dataset["Check_up_year"].astype(str) + "-" + dataset["Month_num"] + "-01")

dataset

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Check_up_month,Check_up_year,Month_num,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81,Februari,2022,02,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10,75,Februari,2022,02,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9,76,Februari,2022,02,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96,Februari,2022,02,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88,Februari,2022,02,2022-02-01
...,...,...,...,...,...,...,...,...,...,...,...
826,46,Sena Humaira Aviaji,04/11/2024,Perempuan,17,8.2,75,Mei,2025,05,2025-05-01
827,47,Nabila Putri Calista,29/05/2024,Perempuan,17,9.1,74.2,Mei,2025,05,2025-05-01
828,48,Kanaya Zunaira Mafaza,18/06/2024,Perempuan,19,8.9,78.1,Mei,2025,05,2025-05-01
829,49,Muh. Davindra Rizky Pratama,29/07/2024,Laki-laki,19,9,73.7,Mei,2025,05,2025-05-01


In [5]:
dataset.drop(columns=['Check_up_month', 'Check_up_year', 'Month_num'], inplace=True)
dataset.to_csv('/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan.csv', index=False)

In [6]:
predict_data = pd.read_csv('/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan.csv')
predict_data.head(10)

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01
5,6,Azqila Mauza Andinitya,24/10/2018,Perempuan,39,21.1,91.0,2022-02-01
6,7,Putri Nadya Silvana,01/10/2019,Perempuan,30,11.9,90.0,2022-02-01
7,8,Dea Fiedella,29/08/2021,Perempuan,6,6.9,65.0,2022-02-01
8,9,Alby Yafie Naditik,06/04/2019,Laki-laki,33,12.5,93.0,2022-02-01
9,10,Angel Almaira,12/06/2019,Perempuan,32,10.7,79.0,2022-02-01


In [7]:
# Load model dari file .h5
model = load_model('/content/drive/MyDrive/Capstone-ML/Capstone/model_stunting.h5')

# Cek ringkasan arsitektur
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 245 (984.00 B)

 Trainable params: 243 (972.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [8]:
predict_data.head()

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01


In [9]:
predict = predict_data.copy()
predict.head()

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01


In [10]:
label = LabelEncoder()
predict['Gender'] = label.fit_transform(predict['Gender'])
predict.head(10)

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,0,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,0,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,0,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,0,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,0,26,10.4,88.0,2022-02-01
5,6,Azqila Mauza Andinitya,24/10/2018,1,39,21.1,91.0,2022-02-01
6,7,Putri Nadya Silvana,01/10/2019,1,30,11.9,90.0,2022-02-01
7,8,Dea Fiedella,29/08/2021,1,6,6.9,65.0,2022-02-01
8,9,Alby Yafie Naditik,06/04/2019,0,33,12.5,93.0,2022-02-01
9,10,Angel Almaira,12/06/2019,1,32,10.7,79.0,2022-02-01


In [11]:
scaler = StandardScaler()
fitur = ['Gender','Age','Height','Weight']
predict[fitur] = scaler.fit_transform(predict[fitur])
predict.head()

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,-1.105407,0.299261,-0.291210,-0.298971,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,-1.105407,-0.750513,-0.377340,-0.719280,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,-1.105407,-1.137272,-0.664439,-0.649229,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,-1.105407,0.244010,0.340408,0.751802,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,-1.105407,-0.198000,-0.262501,0.191390,2022-02-01


In [12]:
X = predict[fitur].values
prediksi = model.predict(X)

predicted_class = np.argmax(prediksi, axis=1)

predict['prediction'] = predicted_class
predict_data['Stunting_Status'] = predict['prediction'].copy()

predict_data.sample(10)

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date,Stunting_Status
83,1,Muhammad Habibie,22/09/2019,Laki-laki,40,11.0,89.0,2022-07-01,0
227,14,Anisa Ramadani,06/06/2020,Perempuan,34,11.4,90.0,2023-03-01,0
448,7,Putri Nadya Silvana,01/10/2019,Perempuan,55,15.8,101.8,2024-03-01,0
378,15,Bahrul Ahmad Al Ghozali,31/03/2018,Laki-laki,68,14.0,100.4,2023-11-01,0
714,34,Velisya Joanna Christanti,29/09/2021,Perempuan,39,12.4,93.5,2025-01-01,0
784,1,Muhammad Habibie,22/09/2019,Laki-laki,73,15.1,104.4,2025-04-01,0
757,16,Cyra Tanisha Nara,11/02/2020,Perempuan,61,18.4,107.2,2025-03-01,0
220,27,Marceline Angelina Monica,23/03/2019,Perempuan,47,10.7,92.0,2023-02-01,2
700,46,Sena Humaira Aviaji,04/11/2024,Perempuan,12,7.4,71.1,2024-12-01,2
539,38,Milan Anggela Kristi,11/10/2023,Perempuan,9,11.1,84.4,2024-06-01,0


In [13]:
label_map = {
    0: 'Normal',
    1: 'Stunting',
    2: 'Severely Stunting'
}

predict_data['Stunting_Status'] = [label_map[i] for i in predict_data['Stunting_Status']]

# Simpan ke
predict_data.to_csv('/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan_with_status.csv', index=False)

In [16]:
predict_data['Stunting_Status'].unique()

array(['Normal', 'Severely Stunting', 'Stunting'], dtype=object)

In [15]:
predict_data.head(10)

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date,Stunting_Status
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01,Normal
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01,Normal
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01,Normal
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01,Normal
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01,Normal
5,6,Azqila Mauza Andinitya,24/10/2018,Perempuan,39,21.1,91.0,2022-02-01,Normal
6,7,Putri Nadya Silvana,01/10/2019,Perempuan,30,11.9,90.0,2022-02-01,Normal
7,8,Dea Fiedella,29/08/2021,Perempuan,6,6.9,65.0,2022-02-01,Normal
8,9,Alby Yafie Naditik,06/04/2019,Laki-laki,33,12.5,93.0,2022-02-01,Normal
9,10,Angel Almaira,12/06/2019,Perempuan,32,10.7,79.0,2022-02-01,Severely Stunting
